In [7]:
import pandas as pd
import numpy as np
import random as rd
import sklearn as sk

wineDF = pd.read_csv('winequality/winequality-red.csv',sep=';')
seedsDF = pd.read_csv('seeds/seeds.csv',sep=',')

class ProcessingData:
    @staticmethod
    def ShuffleData(df: pd.DataFrame):
        for i in range(len(df)):
            rand_i=rd.randint(0,len(df)-1)
            df.loc[i],df.loc[rand_i] = df.loc[rand_i],df.loc[i]
        return df
    @staticmethod
    def SplitData(df:pd.DataFrame,x:int,y:int):
        if x+y>100:
            print("Podano zbyt dużą ilość danych do podziału")
            return
        train_set,test_set = pd.DataFrame(),pd.DataFrame()
        for i in range(len(df)):
            if rd.random()<x/100:
                train_set = train_set.append(df.loc[i])
            else:
                test_set = test_set.append(df.loc[i])
        return train_set,test_set
    @staticmethod
    def NormalizeData(df:pd.DataFrame):
        for atributte in df.columns[:-1]:
            mean = df[atributte].mean()
            stddev = df[atributte].std()
            df[atributte] = (df[atributte]-mean)/stddev
        return df


In [30]:
#TODO Słaby score, poprawić.

class NaiveBayes:
    @staticmethod
    def mean(df:pd.DataFrame,atributte:str):
        return df[atributte].mean()
    @staticmethod
    def stddev(df:pd.DataFrame,atributte:str):
        return df[atributte].std()
    @staticmethod
    def probability(df:pd.DataFrame,atributte:str,value:float,mean:float,stddev:float):
        exponent = np.exp(-(value-mean)**2/(2*stddev**2))
        return (1/(np.sqrt(2*np.pi)*stddev))*exponent
    @staticmethod
    def probability_class(sample:pd.Series,df:pd.DataFrame):
        probability_class = {}
        for Class in df.iloc[:,-1].unique():
            probability_class[Class] = 1/len(df.iloc[:,-1].unique())
            for atributte in sample.index:
                if atributte == df.columns[-1]:
                    continue
                #calculate mean and stddev for atributte and class
                mean = NaiveBayes.mean(df[df.iloc[:,-1]==Class],atributte)
                stddev = NaiveBayes.stddev(df[df.iloc[:,-1]==Class],atributte)
                probability_class[Class] *= NaiveBayes.probability(df[df.iloc[:,-1]==Class],atributte,sample[atributte],mean,stddev)
        return max(probability_class,key=probability_class.get)
    @staticmethod
    def get_accuracy(df:pd.DataFrame,test_set:pd.DataFrame):
        correct = 0
        for i in range(len(test_set)):
            if NaiveBayes.probability_class(df.iloc[i],df) == test_set.iloc[i][-1]:
                correct += 1
        print("Good predicted: ",correct)
        print("Bad predicted: ",len(test_set)-correct)
        return f"{round((correct/len(test_set))*100,2)}%"
   

In [9]:
# class NaiveBayes:
#     @staticmethod
#     def mean(atr):
#         return atr.mean()
#     @staticmethod
#     def stdev(atr):
#         return atr.std()
#     @staticmethod
#     def prob(x,mean,stdev):
#         if stdev == 0:
#             return 0
#         else:
#             exponent = np.exp(-(x-mean)**2/(2*stdev**2))
#             return (1/(np.sqrt(2*np.pi)*stdev))*exponent
#     @staticmethod
#     def classify(train_set: pd.DataFrame, sample):
#         varieties = train_set[train_set.columns[-1]].unique()
#         mean_std = {}
#         probabilities = {}
#         for variety in varieties:
#             mean_std[variety] = {}
#             for atr in train_set.columns:
#                 if atr != train_set.columns[-1]:
#                     mean_std[variety][atr] = [NaiveBayes.mean(train_set[train_set[train_set.columns[-1]] == variety][atr]),NaiveBayes.stdev(train_set[train_set[train_set.columns[-1]] == variety][atr])]
#         for variety in varieties:
#             probabilities[variety] = 1/len(varieties)
#             for atr in sample.index:
#                 if atr != train_set.columns[-1]:
#                     probabilities[variety] *= NaiveBayes.prob(sample[atr],mean_std[variety][atr][0],mean_std[variety][atr][1])
#         return max(probabilities,key=probabilities.get)
#     @staticmethod
#     def accuracy(test_set: pd.DataFrame, train_set: pd.DataFrame):
#         correct = 0
#         for i in range(len(test_set)):
#             if test_set[train_set.columns[-1]].iloc[i] == NaiveBayes.classify(train_set,test_set.iloc[i]):
#                 correct += 1
#         print("Bad predicted: "+str(len(test_set)-correct))
#         return correct/len(test_set)

In [36]:
seedsDF = ProcessingData.NormalizeData(seedsDF)
seedsDF = ProcessingData.ShuffleData(seedsDF)
trainDF,testDF = ProcessingData.SplitData(seedsDF,70,30)
print(NaiveBayes.get_accuracy(trainDF,testDF))

Good predicted:  21
Bad predicted:  38
35.59%
